# **CSI 4142 Assignment 2 - Part 2: Imputation** ##
**Group 119:**\
Sami Hassan,
300169285\
Yassine Sami, 300146704

> # **Introduction**
The goal of this report is to explore three different imputation